This notebook is for collecting a cleaning data for the model.

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from csv import writer
import datetime
import time
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
from lxml import html
import requests
from bs4 import BeautifulSoup, Comment
import re
from unidecode import unidecode
import nltk
import time
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# chrome_options = Options()
# chrome_options.add_extension('/Users/alexstratton2121/Library/Application Support/Google/Chrome/Default/Extensions/gighmmpiobklfepjocnamgkkbiglidom/5.0.2_0.crx')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def isLatin(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [3]:
def get_season_lineups(season):
    # This function gets all the players that played minutes for every team in a given season.
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument("window-size=1280,720")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    
    wait = WebDriverWait(driver, 10)
    driver.get("https://www.basketball-reference.com/leagues/NBA_"+str(season)+".html")
    wait.until(EC.element_to_be_clickable((By.XPATH,
                                           '//*[@id="switcher_per_game_team-opponent"]'))).location_once_scrolled_into_view
    table = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="per_game-team"]')))
    teams = [x.get_attribute('href') for x in table.find_elements(By.TAG_NAME, 'a')]
    lineups = []
    for team in teams:
        r = requests.get(team)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find(id='per_game')
        lineups.append(pd.read_html(table.prettify())[0])
        lineups[len(lineups)-1].iloc[0, 0] = str(team[7:10])
        time.sleep(5)
        
    for lineup in lineups:
        for player in lineup.iloc[:, 1]:
            if isLatin(player) == False:
                player = unidecode(player)
    
    return lineups

In [212]:
season_lineups_2023 = get_season_lineups('2023')

C:\Users\User\AppData\Local\Temp\ipykernel_13700\2456317695.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


[    Rk       Unnamed: 1  Age   G  GS    MP   FG   FGA    FG%   3P  ...    FT%  \
 0    1    Blake Griffin   24  80  80  35.8  9.0  17.0  0.528  0.2  ...  0.715   
 1    2   DeAndre Jordan   25  82  82  35.0  4.2   6.3  0.676  0.0  ...  0.428   
 2    3       Chris Paul   28  62  62  35.0  6.5  14.0  0.467  1.3  ...  0.855   
 3    4   Jamal Crawford   33  69  24  30.3  6.1  14.7  0.416  2.3  ...  0.866   
 4    5      J.J. Redick   29  35  34  28.2  5.2  11.4  0.455  2.1  ...  0.915   
 5    6      Matt Barnes   33  63  40  27.5  3.7   8.4  0.438  1.5  ...  0.733   
 6    7  Darren Collison   26  80  35  25.9  4.1   8.7  0.467  0.9  ...  0.857   
 7    8     Jared Dudley   28  74  43  23.4  2.6   6.0  0.438  1.1  ...  0.655   
 8    9    Danny Granger   30  12   0  16.2  3.0   7.0  0.429  1.0  ...  0.857   
 9   10     Willie Green   32  55   9  15.8  1.9   4.9  0.376  0.7  ...  0.824   
 10  11       Glen Davis   28  23   1  13.4  1.7   3.5  0.481  0.0  ...  0.783   
 11  12  Stephen

In [4]:
def get_lineups(season, month):
    lineups = []
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument("window-size=1280,720")
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    wait = WebDriverWait(driver, 10)
    # Not every season starts/ends in the same month due to Covid. Hence the correct months need to be passed.
    driver.get("https://www.basketball-reference.com/leagues/NBA_"+season+"_games-"+month+".html")

    wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@id="all_schedule"]'))).location_once_scrolled_into_view
    games = wait.until(EC.presence_of_element_located((By.XPATH, '//table[@id="schedule"]')))
    # Getting links to the boxscore of each game
    links = [x.get_attribute('href') for x in games.find_elements(By.TAG_NAME, 'a') if x.text == 'Box Score']
            
    for link in links:
        driver.get(link)
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="line_score"]/tbody/tr[1]/th/a'))).location_once_scrolled_into_view
        teams = [str(driver.find_element(By.XPATH, '//*[@id="line_score"]/tbody/tr[1]/th/a').text),
                 str(driver.find_element(By.XPATH, '//*[@id="line_score"]/tbody/tr[2]/th/a').text)]
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        for team in teams:
            # Getting boxscore and adding team_id into each dataframe.
            table = soup.find(id="box-"+str(team)+"-game-basic")
            lineups.append(pd.read_html(table.prettify())[0])
            lineups[len(lineups)-1].iloc[5, 0] = team
            lineups[len(lineups)-1].iloc[:, 3] = team
            lineups[len(lineups)-1].iloc[:, 2] = datetime.datetime.strptime(str(driver.find_element(
                By.XPATH, '//*[@id="content"]/div[2]/div[3]/div[1]').text), '%H:%M %p, %B %d, %Y').strftime('%Y-%m-%d') + team
            time.sleep(2)
                
        time.sleep(15)
                
    
    # Some players have letters not in the latin alphabet in their name. The dataset I need to match this to \
    # does not have these letters. Therefore I have to find a remove them. Example: 'Nikola Vučević'
    for lineup in lineups:
        for idx, player in enumerate(lineup.iloc[:, 0]):
            if isLatin(player) == False:
                lineup.iloc[idx, 0] = unidecode(player)
                
    
                
    return lineups

In [9]:
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']

In [10]:
lineups_oct = get_lineups('2022', months[0])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [11]:
time.sleep(300)

In [12]:
lineups_nov = get_lineups('2022', months[1])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [13]:
time.sleep(300)

In [14]:
lineups_dec = get_lineups('2022', months[2])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [15]:
time.sleep(300)

In [16]:
lineups_jan = get_lineups('2022', months[3])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [17]:
time.sleep(300)

In [18]:
lineups_feb = get_lineups('2022', months[4])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [19]:
time.sleep(300)

In [20]:
lineups_mar = get_lineups('2022', months[5])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [21]:
time.sleep(300)

In [22]:
lineups_apr = get_lineups('2022', months[6])

C:\Users\User\AppData\Local\Temp\ipykernel_50556\2865946404.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [31]:
lineups = [lineup for month in lineups_2022 for lineup in month]

In [35]:
lineup_df = pd.concat(lineups, axis=0)

In [36]:
lineup_df.head()

Unnamed: 0_level_0 Basic Box Score Stats                                   \
            Starters                    MP             FG  FGA   FG% 3P 3PA   
0         Joe Harris                 30:46  2021-10-19BRK  BRK  .333  3   5   
1       James Harden                 30:38  2021-10-19BRK  BRK  .375  4   8   
2       Kevin Durant                 30:15  2021-10-19BRK  BRK  .520  3   7   
3        Nic Claxton                 24:10  2021-10-19BRK  BRK  .667  0   0   
4      Blake Griffin                 22:59  2021-10-19BRK  BRK  .400  0   1   

                ...                                          
    3P% FT FTA  ... ORB DRB TRB AST STL BLK TOV PF PTS  +/-  
0  .600  0   0  ...   0   2   2   2   0   0   2  1   9    0  
1  .500  4   4  ...   3   5   8   8   1   2   4  3  20  -20  
2  .429  3   6  ...   0  11  11   4   0   2   1  2  32  -20  
3   NaN  0   3  ...   0   7   7   0   0   0   0  3  12  -15  
4  .000  2   2  ...   1   4   5   0   1   0   1  1   6   -6  

[5 rows x 21 columns]

In [38]:
lineup_df.to_excel(r"/Users/User/Data_Collection_Cleaning/lineup_data/lineups_2022.xlsx")

In [23]:
lineups_2022 = [lineups_oct, lineups_nov, lineups_dec, lineups_jan, lineups_feb, lineups_mar, lineups_apr]
lineup_2022_df_raptor = match_past_raptor(lineups_2022, raptor[8])
lineup_2022_df_war = match_past_war(lineups_2022, raptor[8])
lineup_2022_df_raptor.to_excel(r"/Users/User/Desktop/lineup_data/raptor_lineups_2022.xlsx")
lineup_2022_df_war.to_excel(r"/Users/User/Desktop/lineup_data/war_lineups_2022.xlsx")

IndexError: list index out of range

In [6]:
raptor = pd.read_excel(r"/Users/User/Desktop/Programming/raptor_data.xlsx")
raptor = [raptor.loc[raptor['season'] == 2014+i] for i in range(8)]
# for year in raptor:
    # year = year.loc[year['season_type'] == "RS"]
    # year = year.reset_index(drop=True)
    # for player in year.iloc[:, 0]:
        # player = player.replace(" ", "")
        # re.sub(r'[^\w\s]', '', player)

In [7]:
def match_past_raptor(past_lineups, raptor):
    raptor = raptor.loc[raptor['season_type'] == 'RS']
    lineups_dfs = [pd.concat(lineups, axis=0, ignore_index=True) for lineups in past_lineups]
    lineup_df = pd.concat(lineups_dfs, axis=0, ignore_index=True)
    lineup_df.columns = lineup_df.columns.droplevel(0)
    lineup_df.loc[lineup_df.FGA == 'CHO', 'FGA'] = 'CHA'
    
    for idx, player in enumerate(lineup_df.iloc[:, 0]):
        if player in list(raptor.iloc[:, 0]):
            try:
                lineup_df.iloc[idx, 1] = raptor.loc[(raptor['player_name'] == player) & (raptor['team'] == lineup_df.iloc[idx, 3]), 'raptor_total'].item()
            except:
                lineup_df.iloc[idx, 1] = -100
                
    return lineup_df

In [8]:
def match_past_war(past_lineups, raptor):
    raptor = raptor.loc[raptor['season_type'] == 'RS']
    lineups_dfs = [pd.concat(lineups, axis=0, ignore_index=True) for lineups in past_lineups]
    lineup_df = pd.concat(lineups_dfs, axis=0, ignore_index=True)
    lineup_df.columns = lineup_df.columns.droplevel(0)
    lineup_df.loc[lineup_df.FGA == 'CHO', 'FGA'] = 'CHA'
    
    for idx, player in enumerate(lineup_df.iloc[:, 0]):
        if player in list(raptor.iloc[:, 0]):
            try:
                lineup_df.iloc[idx, 1] = raptor.loc[(raptor['player_name'] == player) & (raptor['team'] == lineup_df.iloc[idx, 3]), 'war_reg_season'].item()
            except:
                lineup_df.iloc[idx, 1] = -100
                
    return lineup_df

In [223]:
def match_season(season_lineups, raptor):
    raptor_groups = raptor.groupby(by='team')
    
    for key, players in raptor_groups:
        for team in season_lineups:
            if team.iloc[0, 0] == key:
                for i, player in enumerate(team.iloc[:, 1]):
                    team.iloc[i, 2] = players.loc[players['player_name'] == player, 'raptor_total'].item()
            else:
                continue
    
    return season_lineups

In [217]:
def get_live_lineups():
    # This function gets all of the projected lineups for today's games.
    players = []
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument("window-size=1280,720")
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get("https://rotogrinders.com/lineups/nba?site=draftkings")
    driver.find_element(By.XPATH, '//*[@id="bc-close-cookie"]').click()
    driver.find_element(By.XPATH, '//*[@id="top"]/div/section/div/section/div[1]/div[3]/div[1]/div[3]/div/select').click()

    driver.find_element(By.XPATH, '//*[@id="top"]/div/section/div/section/div[1]/div[3]/div[1]/div[3]/div/select/option[2]').click()
    driver.find_element(By.XPATH, '//*[@id="top"]/div/section/div/section/div[1]/div[3]/div[1]/div[3]/div/select/option[1]').click()
    all_ul = driver.find_elements(By.TAG_NAME, 'ul')
    for ul in all_ul:
        all_li = ul.find_elements(By.TAG_NAME, 'li')
        for li in all_li:
            players.append(li.text)
            
    games_teams = [team.find_elements(By.CLASS_NAME, 'shrt') for team in driver.find_elements(By.CLASS_NAME, 'teams')]
    games_teams = [game for game in games_teams if len(game) != 0]
    for game in games_teams:
        for i, team in enumerate(game):
            game[i] = team.get_attribute("innerHTML")
            
        
    to_keep = ['$', 'Starters', 'Bench']
    current_lineups = []

    for line in players:
        for keep in to_keep:
            if ((line.find(keep) != -1) & (len(line) < 240)):
                current_lineups.append(line)
    # The raptor ratings data for current players has player names in the form: LastFirstLast. This chunk puts today's \
    # projected lineups into the same format.            
    tokens = [word_tokenize(player) for player in current_lineups]
    players = []
    for word in tokens:
        if len(word) == 1:
            players.append(word)
        else:
            players.append(word[0:len(word)-4])
            
    for player in players:
        if len(player) == 2:
            player.insert(0, player[-1])
        elif len(player) == 3:
            player.insert(0, ''.join(player[-2:]))
       
    players = [''.join(player) for player in players]
    players = players[len(players)//2:]
    players = [re.sub(r'[^\w\s]', '', player) for player in players]
    games_teams = [team for game in games_teams for team in game]
    current_raptors = get_current_raptor()
    
    # Here I am creating two elements lists where position 0 is player name and 1 is raptor rating.
    count = 0
    for idx, player in enumerate(players):
        if player == 'Starters':
            players[idx] = games_teams[count]
            count += 1
        else:
            try:
                players[idx] = [player, 
                                     current_raptors.loc[current_raptors['Player'] == player, 'Total_RPTR'].item()]
            except:
                players[idx] = [player, None]
    
    return players, games_teams

In [218]:
live_lineups, teams = get_live_lineups()

C:\Users\User\AppData\Local\Temp\ipykernel_13700\2811388534.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


C:\Users\User\AppData\Local\Temp\ipykernel_13700\4275811214.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [187]:
def get_current_raptor():
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument("window-size=1280,720")
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get('https://projects.fivethirtyeight.com/nba-player-ratings/')
    ratings = [x.get_attribute('data-val') for x in driver.find_elements(By.TAG_NAME, 'td')]
    ratings = [rating for rating in ratings if rating != None]
    ratings_df = pd.DataFrame(np.array(ratings).reshape(int(len(ratings)/14),14))
    
    for idx, player in enumerate(ratings_df.iloc[:, 0]):
        to_remove = [char for char in player if char.isalpha() == False]
        for char in to_remove:
            player = player.replace(char, '')
        
        ratings_df.iloc[idx, 0] = player
        
    ratings_df.rename(columns={0: 'Player', 12: 'Total_RPTR', 13: 'WAR'}, inplace=True)
    
    return ratings_df

In [188]:
current_raptors = get_current_raptor()

C:\Users\User\AppData\Local\Temp\ipykernel_13700\4275811214.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
